<a href="https://colab.research.google.com/github/nickname8888/Quaternions-LPI/blob/neeraj/YoloV4_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YoloV4 implementation 

Before getting started with this code, you must create an annotated dataset and upload the zipped file to google drive. 

In [ ]:
# Mounting the Google Drive to colab. 

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
os.listdir()

In [ ]:
!unzip <ZIP_FILE_LOCATION> -d <LOCATION_TO_BE_UNZIPPED>

In [ ]:
# Cloning the darknet repository for yoloV4

!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Changes to the 'Makefile' to allow training the model. These can also be done manually, but seeing it in a cell is easier. 

%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# Compilation of the model with our Makefile changes. Make sure to run this cell again if the Makefile is updated.

!make

In [ ]:
!ls data/

In [ ]:
# Downloading the pre-trained weights for yolov4. These were trained for 80 classes, on the MSCOCO dataset

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# Checking access to the function. 
# If running on colab the output should look like: 

#  CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
#  CUDNN_HALF=1 
#  OpenCV version: 3.2.0
#  usage: ./darknet detector [train/test/valid/demo/map] [data] [cfg] [weights (optional)]


!./darknet detector

In [ ]:
# Training step
# detector [train/test/valid/demo/map] [data] [cfg] [weights (optional)]

!./darknet detector train data/obj.data cfg/yolov4-custom.cfg /content/drive/My Drive/yolo-4th-Jan-2022/backup/ -dont_show -map

# Testing the model

In [ ]:
# Function to properly read images

def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# Check out the training graph! 

imShow('chart.png')

In [ ]:
# Get the mAP (mean average precision) values for the given dataset.

!./darknet detector map data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/yolo-4th-Jan-2022/backup/yolov4-custom_last.weights -points 0

In [ ]:
# Updating the config file for testing

%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom.cfg

In [ ]:
# Testing the model on custom images

!./darknet detector test data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/yolo-4th-Jan-2022/backup/yolov4-custom_last.weights /content/drive/MyDrive/yolo-4th-Jan-2022/darknet/data/person.jpg -thresh 0.3

imShow('predictions.jpg')